In [2]:
from FINData import FINData
import vectorbtpro as vbt
import numpy as np
from numba import njit
data=FINData.pull(
    ['HPG','VNM'],
    start='2022-01-01'
)

In [3]:
close=data.get("Close")

In [4]:
close

symbol,HPG,VNM
DateTime,,
2022-01-04 00:00:00+00:00,26.80,67.93
2022-01-05 00:00:00+00:00,26.83,67.54
2022-01-06 00:00:00+00:00,26.43,66.91
2022-01-07 00:00:00+00:00,26.26,66.76
2022-01-10 00:00:00+00:00,26.17,66.68
...,...,...
2025-11-20 00:00:00+00:00,27.25,60.70
2025-11-21 00:00:00+00:00,27.40,59.90
2025-11-24 00:00:00+00:00,27.10,63.00


In [5]:
symbol_wrapper=close.vbt.wrapper

In [6]:
symbol_wrapper.shape

(972, 2)

In [7]:
index_ns=vbt.dt.to_ns(close.index)
index_ns

array([1641254400000000000, 1641340800000000000, 1641427200000000000,
       1641513600000000000, 1641772800000000000, 1641859200000000000,
       1641945600000000000, 1642032000000000000, 1642118400000000000,
       1642377600000000000, 1642464000000000000, 1642550400000000000,
       1642636800000000000, 1642723200000000000, 1642982400000000000,
       1643068800000000000, 1643155200000000000, 1643241600000000000,
       1643328000000000000, 1644192000000000000, 1644278400000000000,
       1644364800000000000, 1644451200000000000, 1644537600000000000,
       1644796800000000000, 1644883200000000000, 1644969600000000000,
       1645056000000000000, 1645142400000000000, 1645401600000000000,
       1645488000000000000, 1645574400000000000, 1645660800000000000,
       1645747200000000000, 1646006400000000000, 1646092800000000000,
       1646179200000000000, 1646265600000000000, 1646352000000000000,
       1646611200000000000, 1646697600000000000, 1646784000000000000,
       1646870400000

In [8]:
@njit
def place_func_nb(c, index):
    """
    c: GenEnContext
        - c.out: đoạn output (mask) cần sửa, dài = to_i - from_i
        - c.from_i, c.to_i: đoạn index đang xét (ở đây luôn là cả cột)
        - c.col: index cột hiện tại (0 = HPG, 1 = VNM, ...)
    index: array[int64]
        - index time ở dạng ns (UTC)
    """
    last_i = -1  # vị trí signal cuối cùng tương đối trong c.out

    for out_i in range(len(c.out)):
        # vị trí global trong index
        i = c.from_i + out_i
   
        # Nếu là bar đầu tiên của series -> chắc chắn là first day
        if i == 0:
            c.out[out_i] = True
            last_i = out_i
        else:
            # so sánh month hiện tại với month của bar trước
            month_now = vbt.dt_nb.month_nb(index[i])
            month_prev = vbt.dt_nb.month_nb(index[i - 1])

            # nếu đổi tháng -> đây là ngày giao dịch đầu tiên của tháng mới
            if month_now != month_prev:
                c.out[out_i] = True
                last_i = out_i

    # phải return vị trí signal cuối cùng (tương đối trong c.out)
    return last_i


In [9]:
mask = vbt.pd_acc.signals.generate(
    symbol_wrapper.shape,  # shape output
    place_func_nb,         # hàm placement
    index_ns,              # args truyền vào sau context: (index,)
    wrapper=symbol_wrapper
)


In [10]:
EntryGenerator = vbt.SignalFactory(
    mode="entries",
    input_names=["close"],
    param_names=["pct_th"],
).with_place_func(
    entry_place_func_nb=place_func_nb,
    entry_settings=dict(
        # truyền input "close" vào hàm place_func_nb
        pass_inputs=["close"],
        # truyền param "pct_th" vào hàm
        pass_params=["pct_th"],
    ),
)


In [11]:
from FINData import FINData
import vectorbtpro as vbt
from numba import njit
import numpy as np

data = FINData.pull(['HPG', 'VNM'], start='2014-01-01')
close = data.get("Close")


In [12]:
@njit
def place_func_nb(c, close, pct_th):
    last_i = -1
    for out_i in range(len(c.out)):
        i = c.from_i + out_i
        if i == 0:
            continue
        prev_price = close[i - 1, c.col]
        curr_price = close[i, c.col]
        if prev_price != 0.0:
            ret = (curr_price - prev_price) / prev_price
            if ret >= pct_th:
                c.out[out_i] = True
                last_i = out_i
    return last_i

EntryGenerator = vbt.SignalFactory(
    mode="entries",
    input_names=["close"],
    param_names=["pct_th"],
).with_place_func(
    entry_place_func_nb=place_func_nb,
    entry_settings=dict(
        pass_inputs=["close"],
        pass_params=["pct_th"],
    ),
)

entry_gen = EntryGenerator.run(close, [0.02, 0.03, 0.05])
entries_3pct = entry_gen.entries[0.03]  # chọn pct_th = 3%


100%|##########| 1/1 [00:04<00:00,  4.02s/it]

In [13]:
entries_3pct

symbol,HPG,VNM
DateTime,,
2014-01-02 00:00:00+00:00,False,False
2014-01-03 00:00:00+00:00,False,False
2014-01-06 00:00:00+00:00,False,False
2014-01-07 00:00:00+00:00,False,False
2014-01-08 00:00:00+00:00,False,False
...,...,...
2025-11-20 00:00:00+00:00,False,False
2025-11-21 00:00:00+00:00,False,False
2025-11-24 00:00:00+00:00,False,True


In [14]:
@njit
def exit_place_func_nb(c, wait_n):
    """
    c: GenExContext
      - c.out: đoạn mask exit tương ứng với segment sau 1 entry
      - len(c.out): số bar trong segment (giữa entry hiện tại và entry kế tiếp)
      - c.from_i: index bắt đầu segment (global)
    wait_n: số bar muốn chờ (3, 5, 10, ...)
    """
    # Nếu segment không đủ dài để chờ wait_n bar → không có exit
    if len(c.out) <= wait_n:
        return -1

    # Đặt exit tại bar thứ wait_n trong segment
    out_i = wait_n
    c.out[out_i] = True
    return out_i


In [15]:
wait_n = 5  # thoát sau 5 bar

exits_3pct = entries_3pct.vbt.signals.generate_exits(
    exit_place_func_nb,
    wait_n
)


In [18]:
entries_3pct.sum()
exits_3pct.sum()

symbol
HPG    120
VNM     75
dtype: int64

In [21]:
pf = vbt.Portfolio.from_signals(
    close,
    entries=entries_3pct,
    exits=exits_3pct,
    direction="long_only",
    init_cash=100,
    fees=0.001,
)

print(pf['HPG'].stats())


Start Index                   2014-01-02 00:00:00+00:00
End Index                     2025-11-26 00:00:00+00:00
Total Duration                       2968 days 00:00:00
Start Value                                       100.0
Min Value                                     88.683181
Max Value                                    379.943631
End Value                                    363.534816
Total Return [%]                             263.534816
Benchmark Return [%]                        1099.115044
Position Coverage [%]                         35.411051
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              50.037636
Max Drawdown Duration                1045 days 00:00:00
Total Orders                                        240
Total Fees Paid                               42.579034
Total Trades                                        120
Win Rate [%]                                       55.0
Best Trade [%]                                27

In [22]:
@njit
def exit_place_func_nb(c, wait_n):
    if len(c.out) <= wait_n:
        return -1
    out_i = wait_n
    c.out[out_i] = True
    return out_i


In [23]:
ExitGenerator = vbt.SignalFactory(
    mode="exits",
    param_names=["wait_n"],
).with_place_func(
    exit_place_func_nb=exit_place_func_nb,
    exit_settings=dict(
        pass_params=["wait_n"],  # truyền wait_n vào hàm
    ),
    var_args=True,  # cho phép truyền nhiều param values
)


In [24]:
wait_ns = [3, 5, 10]

exit_gen = ExitGenerator.run(
    entries_3pct,  # input chính là entry mask
    wait_ns        # param wait_n
)


In [25]:
exit_gen.exits.sum()


custom_wait_n  symbol
3              HPG       146
               VNM        81
5              HPG       120
               VNM        75
10             HPG        79
               VNM        62
dtype: int64

In [28]:
exits_wait5 = exit_gen.exits[3]

pf_wait5 = vbt.Portfolio.from_signals(
    close,
    entries=entries_3pct,
    exits=exits_wait5,
    direction="long_only",
    init_cash=100_000_000,
    fees=0.001,
)

pf_wait5['HPG'].stats()


Start Index                   2014-01-02 00:00:00+00:00
End Index                     2025-11-26 00:00:00+00:00
Total Duration                       2968 days 00:00:00
Start Value                                 100000000.0
Min Value                               87935268.328748
Max Value                              330567534.146684
End Value                              322576972.307529
Total Return [%]                             222.576972
Benchmark Return [%]                        1099.115044
Position Coverage [%]                         25.977089
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              30.969033
Max Drawdown Duration                 600 days 00:00:00
Total Orders                                        292
Total Fees Paid                         47341184.933099
Total Trades                                        146
Win Rate [%]                                  49.315068
Best Trade [%]                                43

In [29]:
from FINData import FINData
import vectorbtpro as vbt
from numba import njit
import numpy as np

data = FINData.pull(
    ['HPG', 'VNM'],
    start='2014-01-01'
)

high = data.get("High")
low = data.get("Low")
close = data.get("Close")


In [30]:
@njit
def entry_place_func_nb(c, low, close, entry_th):
    """
    c: GenEnExContext
    low, close: 2D arrays [time, col]
    entry_th: ngưỡng % giảm từ giá thoát gần nhất (ví dụ 0.1 = 10%)
    """
    # Lần đầu tiên (từ index 0) thì ta cho entry ngay tại bar đầu tiên để khởi động chain
    if c.from_i == 0:
        c.out[0] = True
        return 0

    # c.from_i là bắt đầu segment hiện tại (sau 1 exit)
    # c.wait = khoảng cách tối thiểu giữa entry/exit (VBT quản lý)
    # entry segment bắt đầu ngay sau 1 exit, nên exit_i = c.from_i - c.wait
    exit_i = c.from_i - c.wait
    exit_price = close[exit_i, c.col]

    # Giá mục tiêu để buy: giảm entry_th so với exit_price
    hit_price = exit_price * (1 - entry_th)

    for out_i in range(len(c.out)):
        i = c.from_i + out_i
        # Khi low chạm hoặc thấp hơn hit_price → đặt entry tại đây
        if low[i, c.col] <= hit_price:
            return out_i  # không cần set c.out[out_i] = True, VBT sẽ tự đánh dấu
    return -1


In [31]:
@njit
def exit_place_func_nb(c, high, close, exit_th):
    """
    c: GenEnExContext
    high, close: 2D arrays [time, col]
    exit_th: ngưỡng % tăng từ giá vào lệnh (ví dụ 0.2 = 20%)
    """
    # entry segment bắt đầu ngay sau 1 entry, nên entry_i = c.from_i - c.wait
    entry_i = c.from_i - c.wait
    entry_price = close[entry_i, c.col]

    # Giá mục tiêu chốt lời
    hit_price = entry_price * (1 + exit_th)

    for out_i in range(len(c.out)):
        i = c.from_i + out_i
        if high[i, c.col] >= hit_price:
            return out_i
    return -1


In [39]:
BothGenerator = vbt.SignalFactory(
    mode="both",
    input_names=["high", "low", "close"],
    param_names=["entry_th", "exit_th"],
).with_place_func(
    # hàm entry
    entry_place_func_nb=entry_place_func_nb,
    entry_settings=dict(
        pass_inputs=["low", "close"],      # truyền low, close vào hàm entry
        pass_params=["entry_th"],          # truyền entry_th
    ),

    # hàm exit
    exit_place_func_nb=exit_place_func_nb,
    exit_settings=dict(
        pass_inputs=["high", "close"],     # truyền high, close vào hàm exit
        pass_params=["exit_th"],           # truyền exit_th
    ),
)


In [40]:
entry_ths = [0.05, 0.10]
exit_ths = [0.10, 0.15]

both_gen = BothGenerator.run(
    high,
    low,
    close,
    entry_ths,
    exit_ths,
    param_product=True   # tạo grid tất cả các cặp (entry_th, exit_th)
)


100%|##########| 1/1 [00:02<00:00,  2.29s/it]

In [34]:
print(both_gen.entries.sum())
print(both_gen.exits.sum())


custom_entry_th  custom_exit_th  symbol
0.05             0.10            HPG       2
                                 VNM       3
                 0.15            HPG       5
                                 VNM       2
0.10             0.10            HPG       1
                                 VNM       2
                 0.15            HPG       1
                                 VNM       1
dtype: int64
custom_entry_th  custom_exit_th  symbol
0.05             0.10            HPG       2
                                 VNM       3
                 0.15            HPG       5
                                 VNM       2
0.10             0.10            HPG       1
                                 VNM       2
                 0.15            HPG       1
                                 VNM       1
dtype: int64


In [41]:
entries_5_15 = both_gen.entries[(0.05, 0.15)]
exits_5_15   = both_gen.exits[(0.05, 0.15)]

In [42]:
pf_5_15 = vbt.Portfolio.from_signals(
    close,
    entries=entries_5_15,
    exits=exits_5_15,
    direction="long_only",
    init_cash=100_000_000,
    fees=0.001,
)

In [43]:
fig = close["HPG"].vbt.plot()
entries_5_15["HPG"].vbt.signals.plot_as_entries(y=close["HPG"], fig=fig)
exits_5_15["HPG"].vbt.signals.plot_as_exits(y=close["HPG"], fig=fig)
fig.show()